In [1]:
import torch
import cv2
from roboflow import Roboflow
import dotenv
import os
dotenv.load_dotenv()
print("GPU disponible:", torch.cuda.is_available())

!nvidia-smi

GPU disponible: True
Wed Aug  6 18:39:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.169                Driver Version: 570.169        CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   54C    P3             20W /   55W |    1280MiB /   8188MiB |      7%      Default |
|                                         |                        |                  N/A |
+--------------------------

In [5]:
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
project = rf.workspace(os.getenv("ROBOFLOW_WORKSPACE")).project(os.getenv("ROBOFLOW_PROJECT_ID"))
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow project...



Extracting Dataset Version Zip to larc-2 in yolov8:: 100%|██████████| 534/534 [00:00<00:00, 6243.33it/s]



In [7]:
from ultralytics import YOLO

# Cargar el modelo pre-entrenado YOLOv8m
model = YOLO("yolov8m.pt")

# Entrenar el modelo con el dataset descargado de Roboflow
results = model.train(
    data="larc-2/data.yaml",  # Usar el archivo data.yaml del dataset descargado
    epochs=150,
    imgsz=640,
    batch=8,
    split=0.8,  # Dividir automáticamente: 80% entrenamiento, 20% validación
    perspective=0.0005,
    scale=0.4,
    translate=0.04,
    degrees=5,
    shear=1,
    hsv_s=0.5,
    hsv_v=0.3,
    flipud=0.2,
    patience=15,
    save_period=20,
    name="larc-model",
)

# Validar el modelo entrenado
model.val()

Ultralytics 8.3.174 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7817MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=larc-2/data.yaml, degrees=5, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=larc-model2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0005, plots=True, po

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4657.5±1770.8 MB/s, size: 73.0 KB)
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4657.5±1770.8 MB/s, size: 73.0 KB)


train: Scanning /home/hector/Desktop/LARC_OPEN2025/vision/model/larc-2/train/labels... 264 images, 3 backgrounds, 0 corrupt: 100%|██████████| 264/264 [00:00<00:00, 826.50it/s]

train: New cache created: /home/hector/Desktop/LARC_OPEN2025/vision/model/larc-2/train/labels.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4148.8±2984.6 MB/s, size: 64.8 KB)


...

     33/150       4.3G     0.3778     0.3764     0.8555         54        640: 100%|██████████| 33/33 [00:09<00:00,  3.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/17 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  6.26it/s]

                   all        264       1083      0.974      0.987      0.985       0.93



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     34/150       4.3G     0.3656     0.3348     0.8533         49        640:  27%|██▋       | 9/33 [00:02<00:07,  3.17it/s]
     34/150       4.3G     0.3656     0.3348     0.8533         49        640:  27%|██▋       | 9/33 [00:02<00:07,  3.17it/s]


KeyboardInterrupt: 

In [ ]:
model = YOLO('')
results = model('')
annotated_img = results[0].plot()

cv2.imshow("YOLO inference", annotated_img)
cv2.waitKey(5000)
cv2.destroyAllWindows()